In [1]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio
%env AWS_SECRET_ACCESS_KEY=minio123
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000
#%env MLFLOW_S3_ENDPOINT_URL=http://192.168.0.21:9000
#%env AWS_ENDPOINT_URL_S3=http://192.168.0.21:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Búsqueda de mejor modelo e hiperparámetros

Dado nuestro dataset de Rain in Australia, el cual ya pasó por el proceso de ETL y se encuentra en nuestro S3 bucket, vamos a realizar una búsqueda de cual seria el mejor modelo y que hiperparametros usar.

La búsqueda de hiperparametros la haremos usando Optuna y el tracking será realizado mediante MLFlow.

OBS: Para la confección de esta notebook, nos basamos en el tutorial de [MLFlow](https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html).

In [2]:
mlflow_server = "http://localhost:5000"
#mlflow_server = "http://192.168.0.21:5000"

mlflow.set_tracking_uri(mlflow_server)

In [3]:
# Cargamos los datos para realizar nuestro estudio.
# OBS, no vamos a cargar los datos de testing, nada de Data leakage por aquí
X_train =  wr.s3.read_csv("s3://data/final/train/weatherAUS_X_train.csv")
y_train =  wr.s3.read_csv("s3://data/final/train/weatherAUS_y_train.csv")

X_test =  wr.s3.read_csv("s3://data/final/test/weatherAUS_X_test.csv")
y_test =  wr.s3.read_csv("s3://data/final/test/weatherAUS_y_test.csv")

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [4]:
from plots import plot_correlation_with_target, plot_information_gain_with_target

Matplotlib is building the font cache; this may take a moment.


In [5]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
correlation_plot = plot_correlation_with_target(X_train, y_train)
information_gain_plot = plot_information_gain_with_target(X_train, y_train)

## Arrancamos a experimentar

In [6]:
import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

import catboost as cb
import xgboost as xgb

from sklearn.metrics import f1_score

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [7]:
# Creemos el experimento
experiment_id = get_or_create_experiment("Rain in Australia")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S')

1


Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [8]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna
    study = optuna.create_study(direction="maximize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=10, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "Rain in Australia",
            "optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos
    if study.best_params["classifier"] == "CatBoost":
        model = cb.CatBoostClassifier(
            random_state=42, 
            logging_level='Silent',
            iterations=study.best_params["iterations"],
            learning_rate=study.best_params["learning_rate"],
            depth=study.best_params["depth"])
    else:
        model = xgb.XGBClassifier(
            objective='binary:logistic', 
            random_state=42, 
            n_jobs=-1,
            n_estimators=study.best_params["n_estimators"],
            learning_rate=study.best_params["learning_rate"],
            max_depth=study.best_params["max_depth"])

    model = model.fit(X_train, y_train.to_numpy().ravel())

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    f1_score = f1_score(y_test.to_numpy().ravel(), y_pred)
    mlflow.log_metric("test_f1", f1_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="rain_in_australia_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


2024/10/10 22:22:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 0 at: http://localhost:5000/#/experiments/1/runs/3c3a9638574c4f3cb2db109e4b17d14d.
2024/10/10 22:22:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Initial trial 0 achieved value: 0.6484131580039012


2024/10/10 22:23:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 1 at: http://localhost:5000/#/experiments/1/runs/9562043e9e614b3c926d730a19252200.
2024/10/10 22:23:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


Trial 1 achieved value: 0.660823796371387 with  1.8781% improvement


2024/10/10 22:23:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 2 at: http://localhost:5000/#/experiments/1/runs/a5615991595343cf8897cf3a20a42a53.
2024/10/10 22:23:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:25:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 3 at: http://localhost:5000/#/experiments/1/runs/3d077967ab0147f782aaf51e8c4753f6.
2024/10/10 22:25:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:25:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 4 at: http://localhost:5000/#/experiments/1/runs/69ee3be65a934fdfbf1b454079fa4673.
2024/10/10 22:25:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:26:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 5 at: http://localhost:5000

Trial 5 achieved value: 0.6641715861479622 with  0.5041% improvement


2024/10/10 22:28:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 6 at: http://localhost:5000/#/experiments/1/runs/343b14cd545942a08cfc2b79227694ca.
2024/10/10 22:28:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:31:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 7 at: http://localhost:5000/#/experiments/1/runs/336feacc0bf3422cb631708c9e9fe555.
2024/10/10 22:31:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:37:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 8 at: http://localhost:5000/#/experiments/1/runs/72d6ff5168024c57957da63daca798d7.
2024/10/10 22:37:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
2024/10/10 22:39:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial: 9 at: http://localhost:5000

## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [9]:
loaded = mlflow.sklearn.load_model(model_uri)

In [13]:
import numpy as np
test_data = [1.65855392657198,0.30756364773796563,4.986784904935632,0.1629978204652361,-0.9652014543212855,0.6545111599012757,0.9325129961557829,0.17443739988006893,1.526745862383178,2.099522766254367,-1.9822748082814323,-2.143035108089854,1.047121452753144,1.0993061634266754,1.051597524337677,0.4797909215862917,1.8726964071996366,-1.4795470845382583,-1.2863854344175085,-1.444479785238645,1.3499568233777028,-0.4989263146587305,1.2954977403055983,-0.5666289400453428,1.0126487895253307,-0.948484894355671,2.2013545335401483,0.42772111773795907,0.027650040936210932,-1.440909455838103
]
loaded.predict(np.array(test_data).reshape([1, -1]))

array([1])

## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [11]:
from mlflow import MlflowClient

client = MlflowClient()
name = "rain_in_australia_model_prod"
desc = "This classifier detects if it will rain tomorrow in an Australian city"

# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2024/10/10 22:51:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rain_in_australia_model_prod, version 1
